In [15]:
import sys
import csv
import math
from gurobipy import *

In [2]:
#example of how to open a csv file as a dictionary - would be necesssary if I could read in the actual data, but I can't
reader = csv.reader(open('workbook1.csv'))
result1 = {}
#tells it that the first column in a row is the key, and every other entry in the row is the data
for row in reader:
    key = row[0]
    if key in result1:
        pass
#iterates for all rows
    result1[key] = row[1:]
print(result1)
#repeat this process for however many files


{'C': ['7', '8', '9'], 'B': ['4', '5', '6'], 'A': ['1', '2', '3']}


In [3]:
#list of dubious assumptions I had to make to get my code to run:
    #CSVs are of a form easily put into a dictionary (ie there is a key in the first column)
    #This means that all the data files would need to be reworked into a key - data form
    #All faults have an optimal sliprate without error bars
    #All earthquakes can be put on any fault
    #there are a limited number of faults and eathquakes

In [4]:
#Gurobi and dictionaries
#Code Based on
    #https://www.gurobi.com/documentation/7.0/examples/netflow_py.html#subsubsection:netflow.py 
    #http://www.gurobi.com/documentation/7.0/examples/workforce3_py.html
#let optimal sliprate 1 = 5, sliprate 2 = 10, sliprate 3 = 15, sliprate 4 = 20

In [16]:
Fault, Faultslip = multidict({
  ('Fault1'):  5,
  ('Fault2'):  10,
    ('Fault3'):  15,
    ('Fault4'): 20})

#for reading in a file (i.e. reader in cell 2) Faultslip = reader, Sumofslip = sum(reader.values())

Earthquke, Slipearthquake= multidict({
  ('event1'): 6,
  ('event2'): 4,
  ('event3'): 7,
  ('event4'): 3,
  ('event5'): 2,
  ('event6'): 8,
  ('event7'): 5,
  ('event8'): 5,
  ('event9'): 9,
  ('event10'): 1 })
#length of fault required by each earthquake
#I just divided the sliprate by 2 which is obviously incorrect
lengthrequired = {
    ('event1'): 3,
  ('event2'): 2,
  ('event3'): 3.5,
  ('event4'): 1.5,
  ('event5'): 1,
  ('event6'): 4,
  ('event7'): 2.5,
  ('event8'): 2.5,
  ('event9'): 4.5,
  ('event10'): .5}
sumrequired = sum(lengthrequired.values())
print (sumrequired)
Lengthoffault = {
  ('Fault1'):  2.5,
  ('Fault2'):  5,
    ('Fault3'):  7.5,
    ('Fault4'): 10    
}
sumfaultlength = sum(Lengthoffault.values())
print(sumfaultlength)

25.0
25.0


In [19]:
# Create optimization model
m = Model('Exampleearthquakes')

m.setObjective(quicksum(Slipearthquake[w]*x[w,s] for w,s in Faultslip), GRB.MINIMIZE)


# Create variables
x = m.addVars(availability, ub=1, name="x")

reqCts = m.addConstrs((x.sum('*', s) == Faultslip[s]
                      for s in Slipearthquake), "_")
#I would need a length requirement here but I'm not quite sure how to do thst 

ValueError: too many values to unpack (expected 2)

In [9]:
#since there is a lot of error associated with the slip rate
#I included this line to test the code independantly of the previous step
m = Model('Exampleearthquakes')
status = m.status
if status == GRB.Status.UNBOUNDED:
    print('The model cannot be solved because it is unbounded')
    exit(0)
if status == GRB.Status.OPTIMAL:
    solution = m.getAttr('x', flow)
    for h in commodities:
        print('\nOptimal flows for %s:' % h)
        for i,j in arcs:
            if solution[h,i,j] > 0:
                print('%s -> %s: %g' % (i, j, solution[h,i,j]))
if status != GRB.Status.INF_OR_UNBD and status != GRB.Status.INFEASIBLE:
    print('Optimization was stopped with status %d' % status)
    exit(0)
#My example is feasable so the rest of this code will give a massive error (it assums imperfect conditions)

# do IIS
    print('The model is infeasible; computing IIS')
    removed = []

# Loop until we reduce to a model that can be solved
    while True:

        m.computeIIS()
        print('\nThe following constraint cannot be satisfied:')
        for c in m.getConstrs():
            if c.IISConstr:
                print('%s' % c.constrName)
                # Remove a single constraint from the model
                removed.append(str(c.constrName))
                m.remove(c)
                break
        print('')

        m.optimize()
        status = m.status

        if status == GRB.Status.UNBOUNDED:
            print('The model cannot be solved because it is unbounded')
            exit(0)
        if status == GRB.Status.OPTIMAL:
            break
        if status != GRB.Status.INF_OR_UNBD and status != GRB.Status.INFEASIBLE:
            print('Optimization was stopped with status %d' % status)
        exit(0)

print('\nThe following constraints were removed to get a feasible LP:')
print(removed)

Optimization was stopped with status 1
The model is infeasible; computing IIS
Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    0.0000000e+00   0.000000e+00   0.000000e+00      0s
IIS runtime: 0.00 seconds


GurobiError: Cannot compute IIS on a feasible model

In [ ]:
#Attempt2
# Fault capacity constraint
#m.addConstrs((flow.sum('*',i) <= Slipeathquake[i]), "cap"
#for i in Faultslip:
    #m.addConstr((Sumofearthquake <= Faultslip[i]),"slip")


# Flow conservation constraints

#m.addConstrs((quicksum(flow[h,i] for i in Faultslip.select('*',j)) == quicksum(flow[h,j] for j in Faultslip.select(j,'*'))for h in Faults for j in Earthquakes), "node")
#m.addConstrs((flow.sum(h,'*') + Faultslip[h] == flow.sum(h,j,'*')for h in Faults for j in Earthquakes), "conservation")
             #(flow.sum(h,'*',j) + inflow[h,j] == flow.sum(h,j,'*')for h in commodities for j in nodes), "node")
# Compute optimal solution
#m.optimize()

# Print solution below
# + Faultslip[h]